# Trabalho Pŕatico 1 - Computação Gráfica
Feito por Gustavo Leão e Luigi Eliabe

## Preparação

Antes de executar o código é necessário preparar o ambiente de execução, para isso é necessário a instalação do [Qt6.6](https://doc.qt.io/qtforpython-6/quickstart.html#quick-start "Site oficial do Qt") para python, assim como o interpretador de [python](https://www.python.org/downloads/ "Site oficial do Python") que seja maior que a 3.7. Apśo a instalação de toda as dependencias basta executar todas as células.

## Classes

Aqui serão definidas as classes que seram utilizadas no trabalho, elas represetarão os objetos necessário para a criação da janela, como o ponto, a reta, o polígono, a viewport e window do mundo real.

In [ ]:
class Ponto:
    def __init__(self,x,y) -> None:
        self.x = x
        self.y = y
        pass

    def __str__(self) -> str:
        return (f"({self.x},{self.y})")


In [ ]:
class Reta:
    def __init__(self,*pontos) -> None:
        self.pontosReta = []
        for ponto in pontos:
            self.pontosReta.append(ponto)
        pass

    def __str__(self) -> str:
        return '('+','.join(str(e) for e in self.pontosReta)+')'

In [ ]:
class Poligono:

    def __init__(self,*pontos) -> None:
        self.pontosPoligono = []
        for ponto in pontos:
            self.pontosPoligono.append(ponto)
        pass

    
    def __str__(self) -> str:
        return '('+','.join(str(e) for e in self.pontosPoligono)+')'

In [ ]:
class Window:
    def __init__(self,minX,minY,maxX,maxY) -> None:
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        self.pontos = []
        self.pontos.append(Ponto(self.minX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.maxY))
        self.pontos.append(Ponto(self.minX,self.maxY))
        pass

    def __str__(self) -> str:
        
        return (f"({self.minX},{self.minY},{self.maxX},{self.maxY})")

In [ ]:
class Viewport:
    def __init__(self,minX,minY,maxX,maxY,borda) -> None:
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        self.pontos = []
        self.pontos.append(Ponto(self.minX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.maxY))
        self.pontos.append(Ponto(self.minX,self.maxY))

        self.bordas = []
        self.bordas.append(Ponto(self.minX-borda,self.minY-borda))
        self.bordas.append(Ponto(self.maxX+borda,self.minY-borda))
        self.bordas.append(Ponto(self.maxX+borda,self.maxY+borda))
        self.bordas.append(Ponto(self.minX-borda,self.maxY+borda))
        pass


    def __str__(self) -> str:
        
        return (f"({self.minX},{self.minY},{self.maxX},{self.maxY})")

## Leitura do Arquivo XML

Temos em seguida, a classe responsável por fazer a leitura do arquivo de entrada, nela utilizamos a biblioteca ElementTree para poder fazer a identificação dos objetos e as leituras dos atributos contido nas tags do arquivo. Dentro dessa classe é possível ver comentários adicionados por nós, para servir de auxílio sobre os métodos contidos na biblioteca utilizada, e como foi feita as leituras dos valores.

In [ ]:
class LerXML:
    import xml.etree.ElementTree as Et
    
    def __init__(self,file) -> None:
        
        tree = self.Et.parse(file) #Gera uma árvore com o arquivo
        self.raiz = tree.getroot()          #Define a raiz
        borda = 10

        #Percorre as raizes do arquivos, procurando todas as TAGS com esse nome
        #Nesse caso estamos percorrendo para pegar os valores da ViewPort
        for view in self.raiz.findall('viewport/vpmin'):
            vpmin = view.attrib #Atribui os atributos da TAG

        for view in self.raiz.findall('viewport/vpmax'):
            vpmax = view.attrib

        vpminX = float(vpmin.get('x')) #Pega o atributo X
        vpminY = float(vpmin.get('y')) #Pega o atributo Y
        vpmaxX = float(vpmax.get('x'))
        vpmaxY = float(vpmax.get('y'))


        #Vamos pegar agora os valores da Window
        for win in self.raiz.findall('window/wmin'):
            wmin = win.attrib

        for win in self.raiz.findall('window/wmax'):
            wmax = win.attrib

        wminX = float(wmin.get('x'))
        wminY = float(wmin.get('y'))
        wmaxX = float(wmax.get('x'))
        wmaxY = float(wmax.get('y'))


        self.janela = Window(wminX,wminY,wmaxX,wmaxY)            #Criando a instância da Window
        self.viewport = Viewport(vpminX,vpminY,vpmaxX,vpmaxY,borda)    #Criando a instância da Viewport


        #Pegando os valores dos pontos
        self.pontos = [] #Criando um vetor de Pontos
        for pon in self.raiz.findall('ponto'):
            ponto = pon.attrib
            ponX = float(ponto.get('x'))
            ponY = float(ponto.get('y'))
            self.pontos.append(Ponto(ponX,ponY))    #Adicionando os pontos ao valores

        #Pegando valores das Retas
        pontoRetas = [] #Criamos um vetor de pontos da reta
        self.retas = []      #Criamos um vetor com as retas

        for ret in self.raiz.findall('reta'):
            pontoRetas.clear()
            for ponto in ret:
                x = float(ponto.attrib.get('x'))
                y = float(ponto.attrib.get('y'))
                pontoRetas.append(Ponto(x,y)) #salvando os pontos da reta em um vetor
            reta = Reta(*pontoRetas)
            self.retas.append(reta) #salvadno a reta em um vetor

        
        #Pegar os pontos do Poligono
        pontosPoligono = [] #Fazemos o mesmo esquema das retas
        self.poligonos = []
        for poli in self.raiz.findall('poligono'):
            pontosPoligono.clear()
            for ponto in poli:
                x = float(ponto.attrib.get('x'))
                y = float(ponto.attrib.get('y'))
                pontosPoligono.append(Ponto(x,y))
            poligono = Poligono(*pontosPoligono)
            self.poligonos.append(poligono) #Mandamos um vetor de pontos para salvar o poli
    

## Classe para Criar o Arquivo

Após ser feito a leitura do arquivo e pego os valores contido no mesmo, usamos a seguinte classe para criar um arquivo XML com os valores alterados pela transformação de ViewPort. Nela é importada a mesma biblioteca utilizada para ler o arquivo, para poder manipular e alterar os valores dos atributos das TAGS.

In [ ]:
class CriarXML:
    import xml.etree.ElementTree as Et

    def __init__(self,file,pontos,retas,poligonos) -> None:
        arq = self.Et.parse(file) 
        self.raiz = arq.getroot()

        i=0
        for self.ponto in self.raiz.findall('ponto'):
            self.ponto.set('x',str(pontos[i].x))    #Comando para trocar  os atributos 
            self.ponto.set('y',str(pontos[i].y))    #Atributo, valor modificar
            i+=1       #Conta a quantidade de pontos
        i=0
        for self.reta in self.raiz.findall('reta'):
            j=0         #reseta a cada reta, pois uma reta vai ser limitada a 2 pontos
            for h in self.reta.findall('ponto'):
                h.set('x',str(retas[i].pontosReta[j].x))
                h.set('y',str(retas[i].pontosReta[j].y))
                j+=1    #Conta a quantidade de pontos
            i+=1        #Conta a quantidade de retas
            
        i=0
        for self.poli in self.raiz.findall('poligono'):
            j=0         #reseta a cada reta, pois uma reta vai ser limitada a 2 pontos
            for h in self.poli.findall('ponto'):
                h.set('x',str(poligonos[i].pontosPoligono[j].x))
                h.set('y',str(poligonos[i].pontosPoligono[j].y))
                j+=1    #Conta a quantidade de pontos no poli
            i+=1        #Conta a quantidade de poli

        arq.write('NovoArquivo.xml')


## Transformada de Viewport

Essa classe pega os valores obtidos no arquivo e transforma para os valores na viewport.

In [ ]:
class Transformada:

    def __init__(self,win,view) -> None:
        self.win = win
        self.view = view
        pass

    def transformarPonto(self,x,y) -> Ponto:
        self.pontoX = (((x-self.win.minX)/(self.win.maxX-self.win.minX))*(self.view.maxX-self.view.minX))
        self.pontoY = ((1-(y-self.win.minY)/(self.win.maxY-self.win.minY))*(self.view.maxY-self.view.minY))
        return Ponto(self.pontoX,self.pontoY)
    
    def transformarReta(self,pontos) -> Reta:
        pontosTrans = []
        for x in pontos:
            pontosTrans.append(self.transformarPonto(x.x,x.y))
        return Reta(*pontosTrans)
            
    
    def transformarPoli(self,pontos) -> Poligono:
        pontosTrans = []
        for x in pontos:
            pontosTrans.append(self.transformarPonto(x.x,x.y))
        return Poligono(*pontosTrans)
            
    def __str__(self) -> str:
        return (f"Viewport: {self.view}\nWindow: {self.win}")

## Aplicação do QT

Agora utilizaremos do Qt para cria uma janela, da qual será possível escolher um arquivo .xml com as medidas da viewport, da window do mundo real, e as coordenadas dentro da window de cada ponto a ser transformado e traçado na viewport.

In [ ]:
from PySide6 import QtCore, QtWidgets, QtGui

class MyWidget(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()


        self.button = QtWidgets.QPushButton("Open")
        self.button2 = QtWidgets.QPushButton("Save")
        self.cena = QtWidgets.QGraphicsScene()
        self.layout = QtWidgets.QVBoxLayout(self)

        self.layout.addWidget(self.button)
        self.layout.addWidget(self.button2)
        self.view = QtWidgets.QGraphicsView(self.cena)
        self.layout.addWidget(self.view)



        self.button.clicked.connect(self.LerArq)
        self.button2.clicked.connect(self.SaveArq)


    @QtCore.Slot()

    def CriarViewport(self):
        pontosView = self.file.viewport.pontos
        bordas = self.file.viewport.bordas
        view = []
        for x in pontosView:
            y = QtCore.QPointF(x.x,x.y)
            view.append(y)
        
        self.cena.addPolygon(QtGui.QPolygonF(view))

        view.clear()
        for x in bordas:
            y = QtCore.QPointF(x.x,x.y)
            view.append(y)
        self.cena.addPolygon(QtGui.QPolygonF(view))

        #self.cena.addItem(viewPrint) //Aparece mensagem falando que o item foi add


    def CriarPonto(self):
    
        pen = QtGui.QPen(QtGui.Qt.black)
        for x in self.pontosTransformados:
            point = self.cena.addEllipse(x.x, x.y, 2, 2, pen)
            point.setBrush(QtGui.Qt.black)
   
    def CriarReta(self):
        for x in self.retasTransformados:
            p1 = x.pontosReta[0]
            p2 = x.pontosReta[1]
            y = QtCore.QLineF(QtCore.QPointF(p1.x,p1.y),QtCore.QPointF(p2.x,p2.y))
            self.cena.addLine(y)

    def CriarPoli(self):
        view = []
        for x in self.poligonosTransformados:   
            for y in x.pontosPoligono:
                view.append(QtCore.QPointF(y.x,y.y))

        self.cena.addPolygon(QtGui.QPolygonF(view))

        
    def LerArq(self):
        self.arq = QtWidgets.QFileDialog.getOpenFileName()[0]
            
        self.file = LerXML(self.arq)
        trans = Transformada(self.file.janela,self.file.viewport)
        print(trans)

        self.pontosTransformados = []
        self.retasTransformados = []
        self.poligonosTransformados = []

        for x in self.file.pontos:
            self.pontosTransformados.append(trans.transformarPonto(x.x,x.y))

        for retas in self.file.retas:
            self.retasTransformados.append(trans.transformarReta(retas.pontosReta))

        for poli in self.file.poligonos:
            self.poligonosTransformados.append(trans.transformarPoli(poli.pontosPoligono))
        

        MyWidget.CriarViewport(self)
        MyWidget.CriarPonto(self)
        MyWidget.CriarReta(self)
        MyWidget.CriarPoli(self)

    def SaveArq(self):
        CriarXML(self.arq,self.pontosTransformados,self.retasTransformados,self.poligonosTransformados)
        


## Main

Essa sessão viabiliza a comunicação do código com o sistema operacional, gerando a interface gráfica.

In [ ]:
import sys
if __name__ == '__main__':
    
 
    app = QtWidgets.QApplication([])

    widget = MyWidget()
    widget.resize(800, 600)
    widget.show()

   
   

    

    sys.exit(app.exec())

## Observação

Caso a IDE de Jupyter notebook utilizada seja o VSCode, para rodar o código novamente é necessário reiniciar o kernel de execução do jupyter. Isso pode ser feito clicando no botão reiniciar no topo da janela.